## 活性化関数レイヤの実装

### ReLUレイヤ

$$
\begin{equation}
y = \left \{
\begin{array}{l}
x (x \gt 0) \\
0 (x \leqq 0)
\end{array}
\right.
\end{equation}
$$

$$
\begin{equation}
\frac{\partial y}{\partial x} = \left \{
\begin{array}{l}
1 (x \gt 0) \\
0 (x \leqq 0)
\end{array}
\right.
\end{equation}
$$

In [3]:
import numpy as np

class Relu:
    def __init__(self):
        self.mask = None # 順伝搬の入力で0以上のものを覚えておく

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

## Sigmoidレイヤ

$$y = \frac{1}{1 + exp(-x)}$$

乗算、加算ノードの他にexp、除算ノードが必要。

除算ノードは$y = \frac{1}{x}$で、$\partial y / \partial x = - 1 / x^2 = - y^2$となる。

expノードは$y = exp(x)$を表し、微分しても同じになる。

Sigmoidの各項の逆伝搬を計算すると、$\partial L / \partial y \cdot y^2 exp(-x)$となる。

この逆伝搬の式が入力のxと出力のyのみから表されるため、局所計算のノードのグループではなくsigmoidノード一つにまとめることができる。

さらに、$\partial L / \partial y \cdot y^2 exp(-x) = \partial L / \partial y \cdot y(1-y)$

In [6]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
from common.functions import *

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## Affine/Softmaxレイヤの実装

### Affineレイヤ

ニューラルネットワークの順伝搬での重み付き信号の総和を計算するのに、行列の内積を用いた。
幾何学の分野で行列の内積はアフィン変換と呼ばれるため、行列の内積を行う処理をAffineレイヤという名前にする。

In [9]:
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)
Y = np.dot(X, W) + B

X.shape, W.shape, B.shape, Y

((2,), (2, 3), (3,), array([ 1.3605097 ,  1.51454222,  1.72421504]))

Affineレイヤも計算グラフを用いて内積とバイアスの和に計算グラフを局所化し、逆伝搬を求める。

これまでのレイヤはスカラ値が信号だったが、ここでは行列がノード間を伝搬する。

内積部分の逆伝搬は以下の式で表される。

$$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \cdot W^T$$

$$\frac{\partial L}{\partial W} = X^T \cdot \frac{\partial L}{\partial Y}$$

### バッチ版Affineレイヤ

N個のデータをまとめて順伝搬するAffineレイヤを考える。X, X dot W, Yの次元が異なるだけで、計算グラフは同様となる。

逆伝搬の計算も同様に行うことができるが、バイアスの加算に関してはそれぞれのデータの逆伝搬の値がバイアスの要素に集約されるようにする。

In [11]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [12]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [13]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

### Softmax-with-Lossレイヤ

> Softmax関数
> ニューラルネットワークで分類問題を扱う際に出力層の活性化関数に用いる。
> 各項の総和が1になるので、そのまま確率として扱うことができる。
> $$y_k = \frac{exp(a_k)}{\sum_{i = 1}^n exp(a_i)}$$

> 損失関数
> 学習の指標となる関数。2乗和誤差や交差エントロピー誤差がよく用いられる。
> 損失関数を設定するとパラメータの勾配を計算できるようになるため用いられる。

> 2乗和誤差
> $$E = \frac{1}{2} \sum_k (y_k - t_k)^2$$
> 交差エントロピー誤差
> $$E = - \sum_k t_k \log y_k $$

Softmaxレイヤに合わせて損失関数として交差エントロピー誤差も含めて実装する。

この逆伝搬を計算すると、 $y_i - t-i$というキレイな結果になる。Softmaxレイヤの出力と教師ラベルの差分になる。

ニューラルネットワークの学習の目的は、ニューラルネットワークの出力（Softmaxの出力）を教師ラベルに近づけることなので、この逆伝搬の結果は現在のニューラルネットワークと教師ラベルの誤差を素直に表している。

> これは偶然ではなく、逆伝搬がきれいになるように設計されたのが交差エントロピー誤差である。回帰問題において2乗和誤差を恒等関数の損失関数に用いた場合も同じ逆伝搬となり、これも同様の意図がある。

In [17]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
from common.functions import *

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 損失
        self.y = None # softmaxの出力
        self.t = None # 教師データ (one-hot vector)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx